<h1><Center>Serving regularized logistic regression model using Shiny</Center></h1>

# Lesson Goals

<div style="text-align: Justify"> In this lesson, we will refer the saved model and build a shiny app and test the app by uploading test data. </div><a href="Shiny_logistic_regression.html" target="_blank">here</a>

______
# Prerequests 
  1. Install the below mentioned libraries
  2. we will working on R studio

In [ ]:
library(shiny)
library(shinydashboard)
library(shinythemes)

dashboardPage(skin="black",
              dashboardHeader(title=tags$em("Shiny prediction app", style="text-align:center;color:#006600;font-size:100%"),titleWidth = 800),
              
              dashboardSidebar(width = 250,
                               sidebarMenu(
                                 br(),
                                 menuItem(tags$em("Upload Test Data",style="font-size:120%"),icon=icon("upload"),tabName="data"),
                                 menuItem(tags$em("Download Predictions",style="font-size:120%"),icon=icon("download"),tabName="download")
                                 
                                 
                               )
              ),
              
              dashboardBody(
                tabItems(
                  tabItem(tabName="data",
                          
                          
                          br(),
                          br(),
                          br(),
                          br(),
                          tags$h4("With this shiny prediction app, you can upload your data and get back predictions.
                                  The model is a Regularized Logistic Regression that predicts whether microchips from a 
                                  fabrication plant passes quality assurance (QA). During QA, each microchip goes through various 
                                  tests to ensure it is functioning correctly. Suppose you are the product manager of the 
                                  factory and you have the test results for some microchips on two different tests. 
                                  From these two tests, you would like to determine whether the microchips should be accepted or rejected.", style="font-size:150%"),
                          
                          
                          br(),
                          
                          tags$h4("To predict using this model, upload test data in csv format (you can change the code to read other data types) by using the button below.", style="font-size:150%"),
                          
                          tags$h4("Then, go to the", tags$span("Download Predictions",style="color:red"),
                                  tags$span("section in the sidebar to  download the predictions."), style="font-size:150%"),
                          
                          br(),
                          br(),
                          br(),
                          column(width = 4,
                                 fileInput('file1', em('Upload test data in csv format ',style="text-align:center;color:blue;font-size:150%"),multiple = FALSE,
                                           accept=c('.csv')),
                                 
                                 uiOutput("sample_input_data_heading"),
                                 tableOutput("sample_input_data"),
                                 
                                 
                                 br(),
                                 br(),
                                 br(),
                                 br()
                          ),
                          br()
                          
                  ),
                  
                  
                  tabItem(tabName="download",
                          fluidRow(
                            br(),
                            br(),
                            br(),
                            br(),
                            column(width = 8,
                                   tags$h4("After you upload a test dataset, you can download the predictions in csv format by
                                    clicking the button below.", 
                                           style="font-size:200%"),
                                   br(),
                                   br()
                            )),
                          fluidRow(
                            
                            column(width = 7,
                                   downloadButton("downloadData", em('Download Predictions',style="text-align:center;color:blue;font-size:150%")),
                                   plotOutput('plot_predictions')
                            ),
                            column(width = 4,
                                   uiOutput("sample_prediction_heading"),
                                   tableOutput("sample_predictions")
                            )
                            
                          ))
                )))

In [ ]:
library(caret)
library(shiny)
library(LiblineaR)
library(readr)
library(ggplot2)



load("RegularizedLogisticRegression.rda")    # Load saved model

source("featureMapping.R")                         #  a function for feature engineering. 
#  You can include data imputation, data manipulation, data cleaning,
#  feature transformation, etc.,  functions


shinyServer(function(input, output) {
  
  options(shiny.maxRequestSize = 800*1024^2)   # This is a number which specifies the maximum web request size, 
  # which serves as a size limit for file uploads. 
  # If unset, the maximum request size defaults to 5MB.
  # The value I have put here is 80MB
  
  
  output$sample_input_data_heading = renderUI({   # show only if data has been uploaded
    inFile <- input$file1
    
    if (is.null(inFile)){
      return(NULL)
    }else{
      tags$h4('Sample data')
    }
  })
  
  output$sample_input_data = renderTable({    # show sample of uploaded data
    inFile <- input$file1
    
    if (is.null(inFile)){
      return(NULL)
    }else{
      input_data =  readr::read_csv(input$file1$datapath, col_names = TRUE)
      
      colnames(input_data) = c("Test1", "Test2", "Label")
      
      input_data$Label = as.factor(input_data$Label )
      
      levels(input_data$Label) <- c("Failed", "Passed")
      head(input_data)
    }
  })
  
  
  
  predictions<-reactive({
    
    inFile <- input$file1
    
    if (is.null(inFile)){
      return(NULL)
    }else{
      withProgress(message = 'Predictions in progress. Please wait ...', {
        input_data =  readr::read_csv(input$file1$datapath, col_names = TRUE)
        
        colnames(input_data) = c("Test1", "Test2", "Label")
        
        input_data$Label = as.factor(input_data$Label )
        
        levels(input_data$Label) <- c("Failed", "Passed")
        
        mapped = feature_mapping(input_data)
        
        df_final = cbind(input_data, mapped)
        prediction = predict(my_model, df_final)
        
        input_data_with_prediction = cbind(input_data,prediction )
        input_data_with_prediction
        
      })
    }
  })
  
  
  output$sample_prediction_heading = renderUI({  # show only if data has been uploaded
    inFile <- input$file1
    
    if (is.null(inFile)){
      return(NULL)
    }else{
      tags$h4('Sample predictions')
    }
  })
  
  output$sample_predictions = renderTable({   # the last 6 rows to show
    pred = predictions()
    head(pred)
    
  })
  
  
  output$plot_predictions = renderPlot({   # the last 6 rows to show
    pred = predictions()
    cols <- c("Failed" = "red","Passed" = "blue")
    ggplot(pred, aes(x = Test1, y = Test2, color = factor(prediction))) + geom_point(size = 4, shape = 19, alpha = 0.6) +
      scale_colour_manual(values = cols,labels = c("Failed", "Passed"),name="Test Result")
    
  })
  
  
  # Downloadable csv of predictions ----
  
  output$downloadData <- downloadHandler(
    filename = function() {
      paste("input_data_with_predictions", ".csv", sep = "")
    },
    content = function(file) {
      write.csv(predictions(), file, row.names = FALSE)
    })
  
})

# Further reading


1.Review of Modern Logistic Regression Methods with Application to Small and Medium Sample Size Problems [link](https://link.springer.com/chapter/10.1007/978-3-642-17432-2_22).

2.An Improved GLMNET for L1-regularized Logistic Regression [link](https://www.jmlr.org/papers/volume13/yuan12a/yuan12a.pdf).
 
3.Simulation-based Regularized Logistic Regression [link](https://arxiv.org/pdf/1005.3430.pdf)
 
 
 _________________________
# Summary
In this tutorial, you discovered how to  develop shiny app to serve our model. 

Specifically, you learned:

- How to create a shiny app.